In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df.head()

,enc_id,patient_id,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,88346340,2488608,Caucasian,Male,[60-70),NaN,1,2,6,3,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,92001408,52133202,Caucasian,Male,[70-80),[100-125),2,6,1,7,...,No,No,No,No,No,No,No,No,Yes,1
2,169424316,40945509,Caucasian,Female,[70-80),NaN,3,2,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,272987082,38850777,Caucasian,Female,[50-60),NaN,1,1,7,1,...,No,No,No,No,No,No,No,No,Yes,2
4,150600612,72738225,Caucasian,Female,[80-90),NaN,1,6,7,6,...,No,Down,No,No,No,No,No,Ch,Yes,2


In [4]:
df.shape

(71236, 50)

In [7]:
df.isnull().sum()

enc_id                          0
patient_id                      0
race                         1621
gender                          0
age                             0
weight                      68986
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  28178
medical_specialty           34930
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         15
diag_2                        244
diag_3                        989
number_diagnoses                0
max_glu_serum               67515
A1Cresult                   59356
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [29]:
train_df=df.drop(['age','race','gender','weight','payer_code','medical_specialty','enc_id', 'patient_id','admission_type_id',
       'discharge_disposition_id','max_glu_serum', 'A1Cresult','admission_source_id'],axis=1)

In [30]:
train_df.columns

Index(['time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmission_id'],
      dtype='object')

### Data pre-processing

In [31]:
train_df.isnull().sum()

time_in_hospital              0
num_lab_procedures            0
num_procedures                0
num_medications               0
number_outpatient             0
number_emergency              0
number_inpatient              0
diag_1                       15
diag_2                      244
diag_3                      989
number_diagnoses              0
metformin                     0
repaglinide                   0
nateglinide                   0
chlorpropamide                0
glimepiride                   0
acetohexamide                 0
glipizide                     0
glyburide                     0
tolbutamide                   0
pioglitazone                  0
rosiglitazone                 0
acarbose                      0
miglitol                      0
troglitazone                  0
tolazamide                    0
examide                       0
citoglipton                   0
insulin                       0
glyburide-metformin           0
glipizide-metformin           0
glimepir

In [32]:
alt_df=train_df.drop(["diag_1","diag_2","diag_3"],axis=1)

In [33]:
alt_df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,metformin,repaglinide,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,3,54,3,10,0,0,0,5,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,7,50,6,35,2,1,0,9,Steady,No,...,No,No,No,No,No,No,No,No,Yes,1
2,7,49,4,20,2,0,4,9,No,No,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,1,1,1,15,0,0,0,8,Steady,No,...,No,No,No,No,No,No,No,No,Yes,2
4,6,58,2,26,1,0,0,9,No,No,...,No,Down,No,No,No,No,No,Ch,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,6,61,0,14,0,0,0,7,No,No,...,No,No,No,No,No,No,No,No,Yes,1
71232,5,36,0,12,2,0,0,9,No,No,...,No,No,No,No,No,No,No,No,No,2
71233,6,62,6,28,0,0,0,9,No,No,...,No,Steady,No,No,No,No,No,Ch,Yes,2
71234,4,29,1,11,0,0,0,9,No,No,...,No,Steady,No,No,No,No,No,No,Yes,2
